In [27]:
from skimage import io
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [28]:
map_img = io.imread("data/map_nowalls.png")

In [29]:
def create_lidar(x, y, r, map):
    #correzione assi
    x=427-x
    r=r-180

    if map[x][y] != 254:
        return None
    else:
        #crea una instanza lidar artificiale in base ai pixel vicini
        result = look_around(map,x,y,r,int(20/0.05))
        return result

def look_around(map,x,y,r,dist):
    result = {}

    for i in range(720):
        angle = ((i + r) % 720)
        lidar_range = get_range(map, x, y, np.radians(angle/2),dist)
        result["range{}".format(i)] = lidar_range*0.05
        result["angle{}".format(i)] = np.around(np.radians((i-360)/2),decimals=5)  

    return result

def get_range(map,x,y,rad,dist):

    for i in range(dist):
        x_range = np.cos(rad) * i
        y_range = np.sin(rad) * i
        if map[x+int(x_range)][y+int(y_range)] != 254:
            break

    if map[x+int(x_range)][y+int(y_range)] == 0:
        
        return min(np.sqrt((np.power(int(x_range),2))+(np.power(int(y_range),2))), dist)
    
    return dist

In [30]:
instances= []

width = map_img.shape[0]
heigh = map_img.shape[1]

#random position/rotation generator

In [32]:
tot_inst= 4000

while len(instances) < tot_inst:
    i = np.random.randint(0,width)
    j = np.random.randint(0,heigh)
    r = np.random.randint(-360,360)
    result = create_lidar(i,j,r,map_img)
    if result is not None:
        instances.append([result,i*0.05,j*0.05,np.radians(r/2)])
        print(len(instances))

2501
2502
2503
2504
2505
2506
2507
2508
2509
2510
2511
2512
2513
2514
2515
2516
2517
2518
2519
2520
2521
2522
2523
2524
2525
2526
2527
2528
2529
2530
2531
2532
2533
2534
2535
2536
2537
2538
2539
2540
2541
2542
2543
2544
2545
2546
2547
2548
2549
2550
2551
2552
2553
2554
2555
2556
2557
2558
2559
2560
2561
2562
2563
2564
2565
2566
2567
2568
2569
2570
2571
2572
2573
2574
2575
2576
2577
2578
2579
2580
2581
2582
2583
2584
2585
2586
2587
2588
2589
2590
2591
2592
2593
2594
2595
2596
2597
2598
2599
2600
2601
2602
2603
2604
2605
2606
2607
2608
2609
2610
2611
2612
2613
2614
2615
2616
2617
2618
2619
2620
2621
2622
2623
2624
2625
2626
2627
2628
2629
2630
2631
2632
2633
2634
2635
2636
2637
2638
2639
2640
2641
2642
2643
2644
2645
2646
2647
2648
2649
2650
2651
2652
2653
2654
2655
2656
2657
2658
2659
2660
2661
2662
2663
2664
2665
2666
2667
2668
2669
2670
2671
2672
2673
2674
2675
2676
2677
2678
2679
2680
2681
2682
2683
2684
2685
2686
2687
2688
2689
2690
2691
2692
2693
2694
2695
2696
2697
2698
2699
2700


#get path from file

In [ ]:
with open("data/laser_log2303.csv", "r") as data:
    laser_db = pd.read_csv(data, delimiter=";").copy()

In [15]:
trail = []
for i in laser_db.values:
    if i[2] == -3.14159:
        trail.append(i[4:7])

In [ ]:
instances = []
c=0

for i in trail:
    x = int(i[1]//0.05)
    y = int(i[0]//0.05)
    laser_art = create_lidar(x,y,((np.degrees(i[2])*2)),map_img)
    instances.append([laser_art,i[1],i[0],i[2]])
    print(c)
    c+=1


#save instances

In [26]:
with open('datasetnowall.csv', 'w') as f:
    f.write("cnt;time;angle;range;pos_x;pos_y;pos_yaw\n")
    c=1.0
    for item in instances:
        
        for i in range(720):
            if item[0] is not None:
                f.write("{};0;{};{};{};{};{}\n".format(c,item[0]["angle{}".format(i)],item[0]["range{}".format(i)],item[2],item[1],item[3]))
        
        c += 1